In [1]:
from skimage.color import rgb2hsv
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import keras
import seaborn as sns
import pickle as pkl
import cv2
from matplotlib import pyplot
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Lambda
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.layers import SpatialDropout2D
from contextlib import redirect_stdout
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import pandas as pd
import os
import re
import copy
import time
import training_models
import types
import inspect

Using TensorFlow backend.


In [2]:
# double checking to ensure gpu is enabled for training
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
cwd = os.getcwd()

In [4]:
os.listdir(cwd+ '\\color_data')

['count_.csv', 'frames']

In [5]:
target = pd.read_csv(cwd + '\\data\\count_.csv', names=['target'])

In [6]:
target.head()

,target
0,29
1,30
2,35
3,31
4,26


In [7]:
input_ = np.append(pkl.load(open(r'C:\Users\abelp\machine_learning\crowd_count\data\final_input_aj_1.pkl', 'rb')),
                  pkl.load(open(r'C:\Users\abelp\machine_learning\crowd_count\data\final_input_aj_2.pkl', 'rb')), axis=0)

In [8]:
model_location = 'C:\\Users\\abelp\\machine_learning\\crowd_count\\models\\'

In [9]:
os.listdir(model_location)

['alex_net_trasfer.h5',
 'alex_net_trasfer.png',
 'alex_net_trasfer_actual.h5',
 'alex_net_trasfer_actual.png',
 'alex_net_trasfer_BE.h5',
 'alex_net_trasfer_BE.png',
 'alex_net_trasfer_BE_LR.h5',
 'alex_net_trasfer_BE_LR.png',
 'alex_net_trasfer_deep_BE.h5',
 'alex_net_trasfer_deep_BE.png',
 'alex_net_trasfer_deep_BE_LR.h5',
 'alex_net_trasfer_deep_BE_LR.png',
 'alex_net_trasfer_DO.h5',
 'alex_net_trasfer_DO.png',
 'alex_net_trasfer_v2.h5',
 'alex_net_trasfer_v2.png',
 'augmented_samplealex_net_trasfer.png',
 'augmented_samplealex_net_trasfer_BE.png',
 'augmented_samplealex_net_trasfer_BE_LR.png',
 'augmented_samplealex_net_trasfer_deep_BE_LR.png',
 'augmented_samplealex_net_trasfer_v2.png',
 'augmented_samplebasic_CNN_v2_BN.png',
 'augmented_samplebasic_CNN_v2_BN_LR.png',
 'augmented_samplebasic_CNN_v2_DO_LR.png',
 'basic_CNN.h5',
 'basic_CNN.png',
 'basic_CNN_v2.h5',
 'basic_CNN_v2.png',
 'basic_CNN_v2_BN.h5',
 'basic_CNN_v2_BN.png',
 'basic_CNN_v2_BN_LR.h5',
 'basic_CNN_v2_BN_LR.pn

In [10]:
IMG_SIZE = input_.shape[1]
IMG_SIZE2 = input_.shape[2]

In [11]:
models_ = training_models.automated_model_building

In [12]:
target = np.array(target['target'])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(input_, target, test_size=0.2, random_state=42)

In [14]:
X_train.shape

(1600, 480, 640)

In [15]:
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

In [16]:
X_train.shape

(1600, 480, 640, 1)

In [17]:
del input_

In [18]:
val_loss_dict = ['placeholder',100000000]
to_test = []

In [19]:
def data_Generator():
    while True:
        for i in range(0,len(X_train) // 100):
            time.sleep(0.01)
            yield  X_train[i*25:(i+1)*25], y_train[i*25:(i+1)*25]

In [20]:
filter_size = [128, 256]
kernel_size = [(11,11)]
stride_size = [(3,3), (2,2)]

In [21]:
filter_size_2 = [128, 256]
kernel_size_2 = [(8,8), (4,4)]
stride_size_2 = [(2,2), (1,1)]

In [22]:
a = list(product(filter_size, kernel_size, stride_size))

In [23]:
b = list(product(filter_size_2, kernel_size_2, stride_size_2))

In [24]:
to_test = list(product(a,b))

In [25]:
to_test

[((128, (11, 11), (3, 3)), (128, (8, 8), (2, 2))),
 ((128, (11, 11), (3, 3)), (128, (8, 8), (1, 1))),
 ((128, (11, 11), (3, 3)), (128, (4, 4), (2, 2))),
 ((128, (11, 11), (3, 3)), (128, (4, 4), (1, 1))),
 ((128, (11, 11), (3, 3)), (256, (8, 8), (2, 2))),
 ((128, (11, 11), (3, 3)), (256, (8, 8), (1, 1))),
 ((128, (11, 11), (3, 3)), (256, (4, 4), (2, 2))),
 ((128, (11, 11), (3, 3)), (256, (4, 4), (1, 1))),
 ((128, (11, 11), (2, 2)), (128, (8, 8), (2, 2))),
 ((128, (11, 11), (2, 2)), (128, (8, 8), (1, 1))),
 ((128, (11, 11), (2, 2)), (128, (4, 4), (2, 2))),
 ((128, (11, 11), (2, 2)), (128, (4, 4), (1, 1))),
 ((128, (11, 11), (2, 2)), (256, (8, 8), (2, 2))),
 ((128, (11, 11), (2, 2)), (256, (8, 8), (1, 1))),
 ((128, (11, 11), (2, 2)), (256, (4, 4), (2, 2))),
 ((128, (11, 11), (2, 2)), (256, (4, 4), (1, 1))),
 ((256, (11, 11), (3, 3)), (128, (8, 8), (2, 2))),
 ((256, (11, 11), (3, 3)), (128, (8, 8), (1, 1))),
 ((256, (11, 11), (3, 3)), (128, (4, 4), (2, 2))),
 ((256, (11, 11), (3, 3)), (128

In [26]:
to_test[0]

((128, (11, 11), (3, 3)), (128, (8, 8), (2, 2)))

In [27]:
def auto_train_model(model_params):        
    #filepath = model_location + method + '.h5'
    
    history = 0
    model = 0
    
    es_callback = EarlyStopping(monitor='mse', patience=4)
    #checkpoint = ModelCheckpoint(filepath, monitor='val_mse', save_best_only=True, mode='min')
    
    model = models_.model_builder(IMG_SIZE, IMG_SIZE2, model_params)

    model.compile(optimizer='ADAM', loss='mse', metrics=['mse'])
    try:
        history = model.fit_generator(data_Generator(), steps_per_epoch=25, epochs=500,  callbacks=[es_callback], verbose=0, validation_data=(X_test, y_test))
        
        if val_loss_dict[1] > min(history.history['val_mse']):
            val_loss_dict[0] = model_params
            val_loss_dict[1] = min(history.history['val_mse'])

    except:
        print ('ERROR on model: ' + str(model_params))
        
    if history in locals():
        del history
    if model in locals():
        del model

In [28]:
for x in to_test:
    auto_train_model(x)

ERROR on model: ((256, (11, 11), (2, 2)), (256, (8, 8), (1, 1)))
ERROR on model: ((256, (11, 11), (2, 2)), (256, (4, 4), (1, 1)))


In [29]:
val_loss_dict

[((128, (11, 11), (3, 3)), (128, (4, 4), (2, 2))), 15.647869110107422]

In [31]:
with open('output.txt', 'w') as file:  # Use file to refer to the file object
    file.write(str(val_loss_dict))